In [62]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [90]:
# 필터 커널 세팅
kernel = np.ones((1, 1), np.uint8)
kernel2 = np.ones((3, 3), np.uint8)

# 필터 
red1 = np.array([0, 30, 0]) # 밝은 빨강(주황계열)
red2 = np.array([5, 250, 240])

red3 = np.array([155, 50, 0]) # 어두운 빨강(갈색계열)
red4 = np.array([180, 250, 240])

### 원 검출 코드 X

In [92]:
img = cv2.imread('./frame4.png')


img = cv2.resize(img, (img.shape[1]//2, img.shape[0]//2))
# cv2.imshow('img', img)
img1 = img.copy()

blur = cv2.GaussianBlur( img, (3,3), 0 )    
# HSV 영상으로 변환  
hsv = cv2.cvtColor( blur, cv2.COLOR_BGR2HSV ) 

# 색상에 따른 마스크 생성
mask_red1 = cv2.inRange(hsv, red1, red2)
mask_red2 = cv2.inRange(hsv, red3, red4)

# 마스크 결합
mask_red = mask_red1 + mask_red2
# cv2.imshow('mask_red', mask_red)

# 침식 (노이즈제거) -> 팽창(픽셀 살리기)
erosion_mask = cv2.erode(mask_red, kernel, iterations=1)
# cv2.imshow('erosion_mask', erosion_mask)
dilation_mask = cv2.dilate(erosion_mask, kernel2, iterations=1) 
# cv2.imshow('dilation_mask', dilation_mask)

contours, hierarchy = cv2.findContours(dilation_mask, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)

mask_external = np.zeros(dilation_mask.shape, dilation_mask.dtype)
for i in range(len(contours)) :
    if hierarchy[0][i][3] == -1 :
        cv2.drawContours(mask_external, contours, i, 255, -1)
cv2.imshow('mask_external', mask_external)

mask_internal = np.zeros(dilation_mask.shape, dilation_mask.dtype)
for i in range(len(contours)) :
    if hierarchy[0][i][3] != -1 :
        cv2.drawContours(mask_internal, contours, i, 255, -1)
cv2.imshow('mask_internal', mask_internal)
        
numOfLabels, img_label, stats, centroids = cv2.connectedComponentsWithStats(mask_internal)

for idx, centroid in enumerate(centroids) :
    
    if stats[idx][0] == 0 and stats[idx][1] == 0 :
        continue
    if np.any(np.isnan(centroid)) :
        continue
    
    x, y, w, h, area = stats[idx]
    if area > 120 and abs(w-h) < 5 :
        cv2.rectangle(img1, (x-10, y-10), (x+w+10, y+h+10), (0, 0, 255), 1)
        centerX, centerY = int(centroid[0]), int(centroid[1])
        cv2.circle(img1, ( centerX, centerY ), 2, (0, 255, 0), 2)
        # print(x, y, w, h, area)
        
        gray_detected_img = mask_internal[y-10 : y+h+10,  x-10 : x+w+10]
        circles = cv2.HoughCircles(gray_detected_img, cv2.HOUGH_GRADIENT, 1, 100, param1=200, param2=30, minRadius=0, maxRadius=0)
        if circles is not None :
            circles = np.uint16(np.around(circles))
            
            for i in circles[0, :] :
                cv2.circle(img1, ( centerX, centerY ), 2, (0, 255, 0), 2)
                cv2.rectangle(img1, (x-5, y-5), (x+w+5, y+h+5), (0, 0, 255), 2)
           
        
#         # 원 검출 영역
#         detected_img = img1[y : y+h,  x : x+w]
#         gray_detected_img = cv2.cvtColor(detected_img, cv2.COLOR_BGR2GRAY)      
        
#         # 원 검출
#         circles = cv2.HoughCircles(gray_detected_img, cv2.HOUGH_GRADIENT, 1, 100, param1=200, param2=30, minRadius=5, maxRadius=40)

#         if circles is not None :
#             circles = np.uint16(np.around(circles))

#             for i in circles[0, :] :
#                 cv2.circle(img1, ( centerX, centerY ), 2, (0, 255, 0), 2)
#                 cv2.rectangle(img1, (x-5, y-5), (x+w+5, y+h+5), (0, 0, 255), 2)
            
cv2.imshow('img1', img1)
cv2.waitKey()
cv2.destroyAllWindows()

### 원 검출 코드 추가

In [91]:
img = cv2.imread('./frame4.png')


img = cv2.resize(img, (img.shape[1]//2, img.shape[0]//2))
# cv2.imshow('img', img)
img1 = img.copy()

blur = cv2.GaussianBlur( img, (3,3), 0 )    
# HSV 영상으로 변환  
hsv = cv2.cvtColor( blur, cv2.COLOR_BGR2HSV ) 

# 색상에 따른 마스크 생성
mask_red1 = cv2.inRange(hsv, red1, red2)
mask_red2 = cv2.inRange(hsv, red3, red4)

# 마스크 결합
mask_red = mask_red1 + mask_red2
# cv2.imshow('mask_red', mask_red)

# 침식 (노이즈제거) -> 팽창(픽셀 살리기)
erosion_mask = cv2.erode(mask_red, kernel, iterations=1)
# cv2.imshow('erosion_mask', erosion_mask)
dilation_mask = cv2.dilate(erosion_mask, kernel2, iterations=1) 
# cv2.imshow('dilation_mask', dilation_mask)

contours, hierarchy = cv2.findContours(dilation_mask, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)

mask_external = np.zeros(dilation_mask.shape, dilation_mask.dtype)
for i in range(len(contours)) :
    if hierarchy[0][i][3] == -1 :
        cv2.drawContours(mask_external, contours, i, 255, -1)
# cv2.imshow('mask_external', mask_external)

mask_internal = np.zeros(dilation_mask.shape, dilation_mask.dtype)
for i in range(len(contours)) :
    if hierarchy[0][i][3] != -1 :
        cv2.drawContours(mask_internal, contours, i, 255, -1)
cv2.imshow('mask_internal', mask_internal)
        
numOfLabels, img_label, stats, centroids = cv2.connectedComponentsWithStats(mask_internal)

for idx, centroid in enumerate(centroids) :
    
    if stats[idx][0] == 0 and stats[idx][1] == 0 :
        continue
    if np.any(np.isnan(centroid)) :
        continue
    
    x, y, w, h, area = stats[idx]
    if area > 120 and abs(w-h) < 5 :
        # cv2.rectangle(img1, (x-10, y-10), (x+w+10, y+h+10), (0, 0, 255), 1)
        # centerX, centerY = int(centroid[0]), int(centroid[1])
        # cv2.circle(img1, ( centerX, centerY ), 2, (0, 255, 0), 2)
        # print(x, y, w, h, area)
           
        # 원 검출 영역
        detected_img = img1[y-10 : y+h+10, x-10 : x+w+10]
        gray_detected_img = cv2.cvtColor(detected_img, cv2.COLOR_BGR2GRAY)      
        
        # 원 검출
        circles = cv2.HoughCircles(gray_detected_img, cv2.HOUGH_GRADIENT, 1, 100, param1=200, param2=30, minRadius=5, maxRadius=40)

        if circles is not None :
            circles = np.uint16(np.around(circles))

            for i in circles[0, :] :
                cv2.rectangle(img1, (x-10, y-10), (x+w+10, y+h+10), (0, 0, 255), 1)
                cv2.circle(img1, ( centerX, centerY ), 2, (0, 255, 0), 2)
                            
cv2.imshow('img1', img1)
cv2.waitKey()
cv2.destroyAllWindows()